# Website
### https://www.hksuning.com/

***

# Installation
### If you are running this notebook for the first time, please the follow one sell, otherwise, please ignore.

In [ ]:
!pip install pyppeteer
!pip install beautifulsoup4
!pip install nest_asyncio
!pip install asyncio
!pip install lxml

# Importing

In [115]:
# coding=utf-8

from pyppeteer import launch
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup as bs
import re
from dataclasses import dataclass
import xml.etree.ElementTree as et

In [2]:
nest_asyncio.apply()

# Websites that are needed to be crawling

In [3]:
urls = [
        "https://search.hksuning.com/search/list?ci=503692", # url_tablet
        "https://search.hksuning.com/search/list?ci=503694", # url_laptop
        "https://search.hksuning.com/search/list?ci=503304" # url_phone
]

# Return all web pages of each category

In [90]:
async def get_item(categroy: str):
    browser = await launch()
    page = await browser.newPage()
    await page.goto(categroy)
    # pages = await browser.pages()
#     await browser.close()
    return page, browser

# Start crawling

## Get pages of each item page

In [94]:
list_pages = []
list_browsers = []

In [97]:
async def get_page_num(page) -> int:
    content = bs(await page.content(), 'lxml')
    page_num = re.findall(re.compile(r"共\d頁"), content.prettify())[0][1]
    return int(page_num)

In [96]:
for url in urls:
    page, browser = asyncio.get_event_loop().run_until_complete(get_item(url))
    list_pages.append(page)
    list_browsers.append(browser)

RuntimeError: Event loop stopped before Future completed.

In [98]:
page_tablet, browser_tablet = asyncio.get_event_loop().run_until_complete(get_item(urls[0]))

page_num_tablet = asyncio.get_event_loop().run_until_complete(
        get_page_num(page_tablet)
    )
print(f"There are total {page_num_tablet} pages about tablet")

There are total 2 pages about tablet


In [99]:
page_laptop, browser_laptop = asyncio.get_event_loop().run_until_complete(get_item(urls[1]))

page_num_laptop = asyncio.get_event_loop().run_until_complete(
        get_page_num(page_laptop)
    )
print(f"There are total {page_num_laptop} pages about laptop")

There are total 2 pages about laptop


## Debug!!!

In [88]:
page_phone, browser_phone = asyncio.get_event_loop().run_until_complete(get_item(urls[2]))

# page_num_phone = asyncio.get_event_loop().run_until_complete(
#         get_page_num(page_phone)
#     )
print(f"There are total {page_num_phone} pages about mobile phone")

TimeoutError: Navigation Timeout Exceeded: 30000 ms exceeded.

## Next Page

### Have next page button?

In [69]:
async def have_button(page) -> bool:
    
    source = bs(await page.content(), 'lxml').prettify()
    if 'name="ssdln_503692_bottom_pgup02-3" style="display: none;"' in source:
        return False
    else:
        return True

asyncio.get_event_loop().run_until_complete(have_button(page_tablet))

True

### Last page

In [75]:
async def click_last_page(page) ->bool:
    last_btn_selector = "#bottom_pager > div > a.prev"
    try:
        await page.click(last_btn_selector)
        return True
    except:
        return False
        
asyncio.get_event_loop().run_until_complete(click_last_page(page_tablet))

True

### Click next page button

In [178]:
async def click_next_page(page) -> bool:
    next_btn_selector = "#bottom_pager > div > a.next"
    is_next = asyncio.get_event_loop().run_until_complete(have_button(page))

    if is_next:
        await page.click(next_btn_selector)
        return True
    else:
        # print("No next page!")
        return False

## Data storage

### Data Class

In [176]:
@dataclass
class Product:
    product_index: int
    product_type: str
    product_name: str
    prodect_brand: str
    price: float
    avg_rating: float
    total_comment: int
    seller: str
    website: str

@dataclass
class Customer:
    product_index: int
    username: str
    user_rate: float
    rate_date: str
    buyer_comment: str

        
product_type_dict = {
    "平板電腦": "Tablet",
    "手機": "Mobile Phone",
    "手提電腦": "Laptop"
}

all_product = []
all_customer = []

### Save data

In [224]:
async def test():
    u = dict_urls.get('10555754413')
    page_tmp, browser_tmp = asyncio.get_event_loop().run_until_complete(get_item(u))
    await page_tmp.click('#commentNum > a')
    source_tmp = bs(await page_tmp.content(), 'lxml')
#     resolve_data(source)
    return source_tmp

source_tmp = asyncio.get_event_loop().run_until_complete(test())

In [240]:
def resolve_data(source, key):
    
    _product_index = key
    
    type_html = source.select('#pcFourth > div:nth-child(1) > div > ul > li:nth-child(5) > span > a')
    _product_type = product_type_dict[type_html[0].get_text()]
    
    title = source.h1.get_text()
    _product_name = title.replace("自營", "").replace("香港倉", "").strip()
    
    _product_brand = _product_name.split(" ")[0]
    
    text_price = f"{source.find('span', class_='integer').get_text()}.{source.find('span', class_='decimal').get_text()}".strip()
    _price = float(text_price)
    
    tmp_rate = source.select('#appraise > div.rv-wrap > div.rv-container.db > div.rv-rate.rv-bars > div.rv-rate-wrap.clearfix.rv-rate-empty > div.rv-rate-item.rv-rate-score.l > div > p.score > span')
#     _avg_rating = source.select('')
    
    _total_comment = int(source.select('#commentNum > a > span')[0].get_text()[1])
    
    tmp_seller = source.select('#pcFourth > div.wrapper.mt15 > div.procon-side > div.si-intro > div.si-intro-list > dl > dd')[0]
    _seller = tmp_seller.get_text().replace("<dd>", "").replace("</dd>", "").strip()
    
    
    print(tmp_rate)
    

resolve_data(source_tmp, 10555754413)

[]


## URLs

In [72]:
dict_urls = {}

### Get all item url

In [70]:
async def get_url(page):
    source = bs(await page.content(), 'lxml').prettify()

    item_id = re.findall(
            re.compile(r'(?<=//product.hksuning.com/0000000000/)(\d*).html'),
            source
        )
    item_id = list(dict.fromkeys(item_id))
    tmp_dict = {}
    
    for i in item_id:
        url_patten = f"https://product.hksuning.com/0000000000/{i}.html"
        tmp_dict[i] = url_patten
        dict_urls.update(tmp_dict)
        
    if asyncio.get_event_loop().run_until_complete(have_button(page)):
        
        # Click next page button
        asyncio.get_event_loop().run_until_complete(click_next_page(page))
        
        # Get URLs
        asyncio.get_event_loop().run_until_complete(get_url(page))

# asyncio.get_event_loop().run_until_complete(get_url(page_tablet))

### URLs Traversal

In [107]:
async def traversal(page):
    source = bs(await page.content(), 'lxml').prettify()
    
    for key, value in dict_urls.items():
        page, browser = asyncio.get_event_loop().run_until_complete(get_item(value))
        await page.click('#commentNum')
        source = bs(await page.content(), 'lxml')
        resolve_data(source, key)
        
        browser.close()
        
# asyncio.get_event_loop().run_until_complete(traversal())

## Crawl data

In [76]:
async def get_data(page):
    
    # Get URLs
    asyncio.get_event_loop().run_until_complete(get_url(page))
    asyncio.get_event_loop().run_until_complete(traversal(page))
    
    

asyncio.get_event_loop().run_until_complete(get_data(page_tablet))
len(dict_urls)

80

## Main

In [ ]:
async def main():
    for page in pages:
        syncio.get_event_loop().run_until_complete(get_data(page))
        
        
        
    # Close all browsers
    for b in list_browsers:
        await browser.close()
        
asyncio.get_event_loop().run_until_complete(main())